# Прогноз трафика на базовых станциях

Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd

Чтение тренировочного и тестового датасетов

In [ ]:
train = pd.read_csv('train.csv', index_col=0)
test  = pd.read_csv('test_public.csv', index_col=0)

Целевая переменная

In [ ]:
target_value = ['mb']

В качестве предсказания используется среднее значение по всем месяцам для каждой базовой станции и слоя

In [ ]:
means = train.groupby(by=['place_id', 'id2']).aggregate('mean')[target_value]

In [ ]:
means

Функция вычисления предсказания

In [ ]:
def get_prediction(data, means):
    pred = data.join(means, on=['place_id', 'id2'], how='left')
    
    # Для новых пар базовых станций и слоёв предсказывается среднее значение по всем станциям
    pred.mb.fillna(means.mb.mean(), inplace=True)
    
    return pred[['time_idx', 'place_id', 'id2', 'mb']]

Мера ошибки
$$\operatorname{SMAPE}\left(y, \hat{y}\right)=\frac{1}{n}\sum\limits_{i=1}^n\frac{\left|y_i-\hat{y}_i\right|}{\left|y_i+\hat{y}_i+10^{-15}\right|}$$

In [ ]:
def smape(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred) / np.abs(y_true + y_pred + 1e-15))

Вычислить скор на тренировочной выборке

In [ ]:
train_pred = get_prediction(train.drop(columns=target_value), means)

In [ ]:
100 * (1 - smape(train.mb, train_pred.mb))

Сделать предсказание на тестовой выборке

In [ ]:
test_preds = get_prediction(test, means)
test_preds

Сохранить предсказание

In [ ]:
test_preds.to_csv('submit.csv')